In [1]:
from collections import deque
import random
from tqdm import tqdm
from time import sleep
import json
from time import sleep
import gym
import numpy as np
from src.utils import (
    get_env, get_avg_slowdown, get_sjf_action,
    get_possible_actions, finisihed_job_cnt)

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from src.models.pointer_network import pointer_networks

with open("configs/env.json", 'r') as f:
    config = json.load(f)

env = get_env("configs/env.json")

sds = []
cnt = []

observation = env.reset(seq_no=0 % 50)

discount = 1.00
max_job_length = config['max_job_length']
capa = config['n_resource_slot_capacities']
n_resources = len(config['n_resource_slot_capacities'])
input_size = max_job_length * n_resources + 1
state_size = np.prod(observation['machine'].shape)
embedding_size = hidden_size = 32
num_sample_batch = 15


ptr_net = pointer_networks(
    state_size, input_size, embedding_size, hidden_size,
    max_job_length=max_job_length)

adam = optim.RMSprop(ptr_net.parameters(), lr=0.001)
loss = 0
cnt = 0


In [2]:
def get_trajs(model, env, dg=False):
    cc = np.random.randint(0, 100)
    state = env.reset(seq_no=cc)
    ob_obs = []
    ac_actions = []
    _ys = []
    baseline = np.zeros(shape=(num_sample_batch, config['ep_force_stop']), dtype=np.float32)
    batch_size = 16
    ent_mean = []
    for sample_batch in range(num_sample_batch):
        obs = []
        actions = []
        rewards = []
        list_y = []
        state = env.reset(seq_no=cc)
        for t in range(config['ep_force_stop']):
            if dg:
                print(t)
            action, action_indices, ent = model.get_action(state)
            ent_mean.append(ent)
            state2, reward, done, info = env.step(action)
            obs.append(state)
            actions.append(action_indices)
            rewards.append(reward)
            if done:
                break
            state = state2
        disc_vec = np.array(rewards, dtype=float)
        for i in range(1, len(rewards)):
            disc_vec[i:] *= discount
        for i in range(len(rewards)):
            y_i = np.sum(disc_vec[i:]) / (discount ** i)
            list_y.append(y_i)
        _ys.append(list_y)
        baseline[sample_batch, :len(list_y)] = list_y
        ob_obs.append(obs)
        ac_actions.append(actions)

    baseline = np.mean(baseline, axis=0)
    advs = []
    for sb in range(num_sample_batch):
        ys = np.array(_ys[sb])
        ADV = ys - baseline[:len(ys)]
        advs.append(ADV)
    return [item for sublist in ob_obs for item in sublist], np.hstack(ac_actions), np.hstack(advs)

In [4]:
exe = ThreadPoolExecutor(max_workers=8)

In [5]:
envs = [get_env("configs/env.json") for i in range(8)]

In [ ]:
for iter in range(1000):
    adam.zero_grad()
    loss = 0
    cnt = 0
    fts = []
    for jj in range(8):
        fts.append(
            exe.submit(get_trajs, ptr_net, envs[jj]))
    loss = 0
    cnt = 0
    adam.zero_grad()
    for future in as_completed(fts):
        obs, acs, advs = future.result()
        for ob, ac, adv in zip(obs, acs, advs):
            if len(ac) > 0:
                t_loss = ptr_net.train_single(ob, ac, torch.as_tensor(adv, dtype=torch.float32))
                loss += t_loss
                cnt += 1
    loss /= cnt
    loss.backward()
    adam.step()
    print("loss : ", loss.detach().numpy())
    if (iter >= 0) and (iter % 5 == 0):
        usages = np.empty(shape=(0, 3))
        ep_lengths = []
        slowdowns = []
        rews = []
        ents = []
        fjc = []
        for i in range(30):
            s = env.reset(seq_no=100 + i)
            rr = []
            action = []
            for ep_len in range(config['ep_force_stop']):
                action, action_indices, entropy = ptr_net.get_action(s, argmax=False)
                usage = (100 - env.machine.avbl_slot[0]) / 100.0
                usages = np.vstack([usages, usage])
                s2, r, done, info = env.step(action)
                ents.append(entropy)
                rr.append(r)
                if done:
                    break
                s = s2
            fjc.append(finisihed_job_cnt(info))
            rews.append(np.mean(rr))
            slowdowns.append(get_avg_slowdown(info))

        print("[iter %d] avg rewards: %0.3f\tavg slowdowns: %0.3f, etnropies %0.2f, fjc: %d" %
                (iter, np.mean(rews), np.mean(slowdowns), np.mean(ents), np.sum(fjc)))
        print("avg usages", np.mean(usages, axis=0))

loss :  4.6548853
[iter 0] avg rewards: -5.045	avg slowdowns: 5.806, etnropies 0.69, fjc: 1337
avg usages [0.40674505 0.4062005  0.42433168]
loss :  2.3470194
loss :  1.8873485
loss :  3.7062366
loss :  3.7135303
loss :  2.1692045
[iter 5] avg rewards: -4.484	avg slowdowns: 4.812, etnropies 0.55, fjc: 1337
avg usages [0.39820275 0.39766963 0.41542003]
loss :  0.23871182
loss :  2.813322
loss :  2.1824841
loss :  0.33193547
loss :  0.70885754
[iter 10] avg rewards: -4.090	avg slowdowns: 4.099, etnropies 0.41, fjc: 1337
avg usages [0.39756048 0.39702823 0.41475   ]
loss :  1.6597219
loss :  0.13069738
loss :  0.5466099
loss :  -0.4601334
loss :  0.026335666
[iter 15] avg rewards: -4.080	avg slowdowns: 4.032, etnropies 0.40, fjc: 1337
avg usages [0.40275735 0.40221814 0.42017157]
loss :  -0.040803242
loss :  -0.35644755
loss :  -0.088743284
loss :  0.043900866
loss :  0.54006535
[iter 20] avg rewards: -4.042	avg slowdowns: 3.973, etnropies 0.34, fjc: 1337
avg usages [0.39981752 0.39928224